Resource: https://www.wikiart.org/

* 爬作者

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import os
from tqdm import tqdm
import urllib.request
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


In [3]:
# Primitivism 在 wikiart title 是 na-ve-art-primitivism
# Mannerism 在 wikiart title 是 mannerism-late-renaissance

BASE_URL = 'https://www.wikiart.org/en/'
def convert_url(name):
    if name == "Albrecht Dürer":
        name = 'Albrecht Durer'
    name = name.lower()
    name = name.replace(' ','-')
    return BASE_URL + name + "/all-works"

def convert_name(name):
    if name == "Albrecht Dürer":
        name = 'Albrecht Durer'
    name = name.lower()
    name = name.replace(' ','_')
    name = name.replace('-','_')
    return name
    # print(title)
    # if(title == 'Primitivism'):
    #     return BASE_URL + 'na-ve-art-primitivism'
    # elif (title == 'Mannerism'):
    #     return BASE_URL + 'mannerism-late-renaissance'
    # else:
    #     # 兩字節的 title 空格換成 -
    #     if(len(title.split(' ')) == 2):
    #         title = title.replace(' ', '-')
    #     return BASE_URL + title

In [4]:

artists = pd.read_csv('../artists.csv',usecols=['name', 'genre','paintings'])
artists['url'] = artists['name'].apply(convert_url)
artists['name'] = artists['name'].apply(convert_name)
artists
# for i in range(len(artists)):
#     temp_data = artists.loc[i,'genre']
#     temp_data = temp_data.split(',')
#     if(len(temp_data) <= 1):
#         pass
#     else:
#         artists.drop(index=i, inplace=True)
# artists_crawl_df = artists.groupby('genre').sum().sort_values(by='paintings',ascending=False)
# drop_genre = ['Neoplasticism', 'Byzantine Art', 'Renaissance','Suprematism']
# artists_crawl_df = artists_crawl_df[ ~artists_crawl_df.index.isin(drop_genre)].reset_index()
# artists_crawl_df['crawl_url'] = artists_crawl_df['genre'].apply(convert_url)
# artists_crawl_df

,name,genre,paintings,url
0,amedeo_modigliani,Expressionism,193,https://www.wikiart.org/en/amedeo-modigliani/a...
1,vasiliy_kandinskiy,"Expressionism,Abstractionism",88,https://www.wikiart.org/en/vasiliy-kandinskiy/...
2,diego_rivera,"Social Realism,Muralism",70,https://www.wikiart.org/en/diego-rivera/all-works
3,claude_monet,Impressionism,73,https://www.wikiart.org/en/claude-monet/all-works
4,rene_magritte,"Surrealism,Impressionism",194,https://www.wikiart.org/en/rene-magritte/all-w...
5,salvador_dali,Surrealism,139,https://www.wikiart.org/en/salvador-dali/all-w...
6,edouard_manet,"Realism,Impressionism",90,https://www.wikiart.org/en/edouard-manet/all-w...
7,andrei_rublev,Byzantine Art,99,https://www.wikiart.org/en/andrei-rublev/all-w...
8,vincent_van_gogh,Post-Impressionism,877,https://www.wikiart.org/en/vincent-van-gogh/al...
9,gustav_klimt,"Symbolism,Art Nouveau",117,https://www.wikiart.org/en/gustav-klimt/all-works


In [5]:
# 先自己創立 ROOT_FOLDER
ROOT_FOLDER_PATH = r'C:\Users\Stephanie Chung\Documents\NTUIM 2022 Fall\DLA\Final Project\Wikiart_webcrawl\artists_webcrawl_data'

for i in range(0,50):
    genre = artists.loc[i,'name']
    # url = artists.loc[i,'url']
    # print(url)
    
    # 若不存在該種類則建立此資料夾
    if not os.path.exists(os.path.join(ROOT_FOLDER_PATH, genre)):
        os.makedirs(os.path.join(ROOT_FOLDER_PATH, genre))

## 開始爬蟲

In [90]:
idd = 49
url = artists.loc[idd,'url']
store_folder = artists.loc[idd,'name']
p = artists.loc[idd,'paintings']
print(store_folder)
print('paintings = ', p)
options = EdgeOptions()
options.use_chromium = True
#options.add_argument("-inprivate")
options.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
capabilities = DesiredCapabilities.EDGE

driver = Edge(executable_path = r"C:\Users\Stephanie Chung\Desktop\msedgedriver.exe", options = options,desired_capabilities=capabilities) # Modify the path here...

driver.get(url) 

for num in range(0,8):
    check_height = driver.execute_script("return document.body.scrollHeight;") 
    current_height = driver.execute_script("return window.pageYOffset + window.innerHeight")
    print(num)
    # print(height)
    # 因為 lazy loading，所以要慢慢滑讀資料
    for i in range(int(current_height),check_height):
        driver.execute_script(f"window.scrollTo({current_height},{i})")

    # download the image
    soup = BeautifulSoup(driver.page_source, "html.parser")
    data = soup.find(class_="masonry-content").find('ul').find_all('li')
    for img in data:
        img_src = img.find('img').get('src')
        normal_img_src = img_src.split('!')[0]
        save_src = normal_img_src.split('/')[-1]
        save_path = fr'./genre_data/{store_folder}/' + str(save_src)
        # urllib.request.urlretrieve(normal_img_src, save_path)
        # print(path)
    driver.find_elements(By.XPATH, "//a[@class='masonry-load-more-button']")[0].click()
    time.sleep(10)

jackson_pollock
paintings =  24


C:\Users\STEPHA~1\AppData\Local\Temp/ipykernel_12168/1908545721.py:13: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(executable_path = r"C:\Users\Stephanie Chung\Desktop\msedgedriver.exe", options = options,desired_capabilities=capabilities) # Modify the path here...


In [91]:
soup = BeautifulSoup(driver.page_source, "html.parser")
data = soup.find(class_="masonry-content").find('ul').find_all('li')
len(data)

90

In [92]:
cnt = 0
PATH = r"C:\Users\Stephanie Chung\Documents\NTUIM 2022 Fall\DLA\Final Project\Wikiart_webcrawl\temp"
for img in tqdm(range(len(data))):
    img_src = data[img].find('img').get('src')
    normal_img_src = img_src.split('!')[0]
    save_src = normal_img_src.split('/')[-1]
    
    
    save_path = PATH + "/" + store_folder + "/" + str(cnt) + '.jpg'
    # print(save_path)
    # break
    try:
        urllib.request.urlretrieve(normal_img_src, save_path)
        cnt += 1
    except:
        print('error: ',img)
        continue
        # print(path)
    # break

100%|██████████| 90/90 [07:00<00:00,  4.67s/it]
